<a href="https://colab.research.google.com/github/prateekjoshi565/textrank_text_summarization/blob/master/TestRank_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nhóm 4: Vũ Hữu Tùng (19522497) - Võ Minh Trí (19522396) - Nguyễn Thị Mai Phương (19522064)**

*Tóm Tắt Văn Bản Tiếng Việt - Tóm Tắt bằng Text Rank*

# Lấy dữ liệu

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
%%capture

!wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
!unzip 'master.zip'

In [ ]:
%%capture
# Install the vncorenlp python wrapper
!pip install vncorenlp
# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

In [ ]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *

In [ ]:
# Get Path
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('/content/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('/content/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('/content/vietnews-master/data/test_tokenized/*')

In [ ]:
# Read content of each path file
def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ' '.join(''.join(rows[4:]).split('\n'))
    summary = ' '.join(rows[2].split('\n'))
          
  return {'file' : pathfile,
          'original': original, 
          'summary': summary}

In [ ]:
# create dataframe for each set
def get_dataframe(pathfiles):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    data = executor.map(read_content, pathfiles)
  
  # Make blank dataframe
  data_df = list()
  for d in data:
    data_df.append(d)
  data_df = pd.DataFrame(data_df)
  data_df.dropna(inplace = True)
  data_df = data_df.sample(frac=1).reset_index(drop=True)

  return data_df

In [ ]:
# create dataframe
train_df = get_dataframe(train_paths)
test_df = get_dataframe(test_paths)
val_df = get_dataframe(val_paths)

# Đọc W2V

In [ ]:
!gdown --id '17awjPrJRsTX_zXkhnnA483uTjZ-Tn5Xg'

Downloading...
From: https://drive.google.com/uc?id=17awjPrJRsTX_zXkhnnA483uTjZ-Tn5Xg
To: /content/Files.zip
100% 198M/198M [00:01<00:00, 194MB/s]


In [ ]:
!unzip Files.zip

Archive:  Files.zip
   creating: Files/
  inflating: Files/desktop.ini       
  inflating: Files/test_viet_news    
  inflating: Files/train_viet_news   
  inflating: Files/val_viet_news     
  inflating: Files/W2V_ner.vec       


In [ ]:
# Read embedding
word_dict = []
embeddings_index = {}
embedding_dim = 300
max_feature = len(embeddings_index) + 2

f = open('/content/Files/W2V_ner.vec')
for line in f:
    values = line.split(' ')
    word = values[0] 
    word_dict.append(word)
    try:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except Exception as e:
        pass
f.close()

print('Embedding data loaded')

Embedding data loaded


# Using Text Rank for Summarization in Test Set

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
df_sumary = []
# split the the text in the articles into sentences
sentences = []

for s in test_df.original:
  sentences.append(sent_tokenize(s)) # Tách câu
for i in range(len(test_df.original)):
  try:
    sentences_index = sentences[i]
    clean_sentences = pd.Series(sentences_index).str.replace("[^\w\s]", " ") # Loại bỏ ký tự thường
    for i in range(len(clean_sentences)):
      clean_sentences[i] = " ".join(clean_sentences[i].split())
    clean_sentences = clean_sentences.replace('', np.nan).dropna()


    sentence_vectors = []
    for i in clean_sentences:
      if len(i) != 0:
        v = sum([embeddings_index.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001) # Lấy trung bình của tổng vector câu để tạo ra vector đại diện
      else:
        v = np.zeros((300,))
      sentence_vectors.append(v)

    len(sentence_vectors)

    # Similarity Matrix
    sim_mat = np.zeros([len(sentences_index), len(sentences_index)])
    for i in range(len(sentence_vectors)):
      for j in range(len(sentence_vectors)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]


    # Create Graph
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph) # tính score

    # Calculate Text Rank Score and Sorted after Calculating
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences_index)), reverse=True)

    # Specify number of sentences to form the summary
    sn = 3

    # Generate summary from Top 3 Text Rank Score Highest
    list_result = []
    str1 = []
    for i in range(sn):
      list_result.append(ranked_sentences[i][1])
      str1 = ''.join(str(e) for e in list_result)
      print(ranked_sentences[i][1])  
    df_sumary.append(list_result)
  except:
    list_result = ['']
    df_sumary.append(list_result)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Đó cũng là nhận_định của giới quan_sát về cuộc phản_đòn của chính_quyền ông Trump .
Ông ta tự làm hổ_thẹn chính mình , làm hổ_thẹn đất_nước và toàn_bộ cộng_đồng tình_báo " .
Ông Trump dẫn lời Bongino cáo_buộc " Brennan đã bắt_đầu cuộc điều_tra thất_bại toàn_diện về Tổng_thống Trump .
là đại_uý đang công_tác tại Ban chỉ_huy Quân_sự Quận 3 .
cất_giữ trong tủ quần_áo .
Trong quá_trình trò_chuyện uống nước , em H. mở cặp lấy ra một khẩu súng quân_dụng K54 đưa cho các bạn xem .
Tổng_thống Trump bên ngoài Nhà_Trắng hôm 1/6 . "
Tổng_thống Trump bên ngoài Nhà_Trắng hôm 1/6 .
Tôi hy_vọng cuộc gặp lãnh_đạo Triều_Tiên ở Singapore sẽ khởi_đầu một điều gì đó lớn_lao .
Để có được các sản_phẩm ấy , Triều_Tiên đã tiến_hành hàng_loạt các hoạt_động để tiếp_cận với công_nghệ Mỹ và Hàn_Quốc , theo Recorded_Future .
Từ năm 2002 , Mỹ thực_tế đã xuất_khẩu 484.000 máy_tính và thiết_bị điện_tử sang Triều_Tiên .
Tạp_chí Fortune dẫn báo_cáo của công_ty tìn

In [ ]:
# Make dataframe of result summarization
df = pd.DataFrame(df_sumary, columns = ['predict1', 'predict2','predict3'])
list_df = []
for i in range(len(df.predict1)):
  temp = str(df.predict1[i]) + str(df.predict2[i])+ str(df.predict3[i])
  list_df.append(temp)

temp = df.predict1[1] + df.predict2[1]+ df.predict3[1]
list_df.append(temp)

list_df = pd.DataFrame(list_df, columns = ['predict'])

In [ ]:
list_df.head(3)

,predict
0,"Độc_giả cân_nhắc trước khi xem .Được xem là "" ..."
1,“ Tôi thực_sự thất_vọng khi họ không nhìn thấy...
2,"Ảnh : Kh .Đến khoảng 13h , khi dọn cây lồ_ô , ..."


In [ ]:
# Concat original text and summarization text
bigdata = pd.concat([list_df.reset_index(drop=True), test_df.original.reset_index(drop=True)], axis=1)
bigdata = pd.concat([test_df['file'].reset_index(drop=True), bigdata.reset_index(drop=True)], axis=1)

In [ ]:
bigdata.head(5)

,file,predict,original
0,/content/vietnews-master/data/test_tokenized/0...,"Độc_giả cân_nhắc trước khi xem .Được xem là "" ...",Wu_Yongning nằm trên nóc một toà nhà ở Trung_Q...
1,/content/vietnews-master/data/test_tokenized/0...,“ Tôi thực_sự thất_vọng khi họ không nhìn thấy...,Gián đã là một mối phiền_toái trong gia_đình K...
2,/content/vietnews-master/data/test_tokenized/0...,"Ảnh : Kh .Đến khoảng 13h , khi dọn cây lồ_ô , ...",Cơ_quan_chức_năng điều_tra hiện_trường . Ảnh :...
3,/content/vietnews-master/data/test_tokenized/0...,"Ở tuổi 42 , anh vẫn luôn xuất_hiện trên sân_kh...",Mỹ_Tâm Nữ ca_sĩ Mỹ_Tâm đã có gần 20 năm gắn_bó...
4,/content/vietnews-master/data/test_tokenized/0...,"Đất_nước này nằm trên "" vành_đai lửa "" Thái_Bì...","Một nhà_thờ Hồi_giáo tại thành_phố Palu , tỉnh..."


## Kết quả và đánh giá

In [ ]:
!pip install -q rouge_score

In [ ]:
from rouge_score import rouge_scorer
score_list = []
for i in range(len(bigdata.original)):
  scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
  scores = scorer.score(str(bigdata.predict[i]),str(bigdata.original[i]))
  score_list.append(scores)

In [ ]:
df_score_test = pd.DataFrame(score_list, columns = ['score'])

In [ ]:
score_r1 = []
score_rl = []
score_r2 = []
for i in range(len(score_list)):
  score_r1.append(score_list[i]['rouge1'].fmeasure)
  score_r2.append(score_list[i]['rouge2'].fmeasure)
  score_rl.append(score_list[i]['rougeL'].fmeasure)

In [ ]:
import statistics
print(statistics.mean(score_r1))
print(statistics.mean(score_r2))
print(statistics.mean(score_rl))

0.45200444554247593
0.447521506032258
0.3774385023995794


- **2 câu**
- R1: 0.34111518169308247
- R2: 0.33769186213340274
- RL: 0.3080631855000541
- **3 câu**


- R1: 0.45200444554247593
- R2: 0.447521506032258
- RL: 0.3774385023995794 

# Using Text Rank for Summarization in Train Set

In [ ]:
import networkx as nx
df_train_sumary = []
# split the the text in the articles into sentences
sentences = []

for s in train_df.original:
  sentences.append(sent_tokenize(s))  
for i in range(len(train_df.original)):
  try:
    sentences_index = sentences[i]

    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences_index).str.replace("[^\w\s]", " ")
    for i in range(len(clean_sentences)):
      clean_sentences[i] = " ".join(clean_sentences[i].split())
    clean_sentences = clean_sentences.replace('', np.nan).dropna()


    sentence_vectors = []
    for i in clean_sentences:
      if len(i) != 0:
        v = sum([embeddings_index.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((300,))
      sentence_vectors.append(v)

    len(sentence_vectors)

    # similarity matrix
    sim_mat = np.zeros([len(sentences_index), len(sentences_index)])
    for i in range(len(sentence_vectors)):
      for j in range(len(sentence_vectors)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]



    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences_index)), reverse=True)

    # Specify number of sentences to form the summary
    sn = 1

    # Generate summary
    list_result = []
    str1 = []
    for i in range(sn):
      list_result.append(ranked_sentences[i][1])
      str1 = ''.join(str(e) for e in list_result)
      print(ranked_sentences[i][1])  
    df_sumary.append(list_result)
  except:
    list_result = ['']
    df_sumary.append(list_result)

Streaming output truncated to the last 5000 lines.
Đại_diện Sở_giao_dịch Chứng_khoán Hà_Nội HNX cho biết , VIB là đơn_vị thứ 434 có_mặt trên sàn UPCoM .
Đối_với cá_nhân , vi_phạm hành_vi nói trên sẽ bị phạt tiền từ 250.000 đồng đến 500.000 đồng .
Để chữa bệnh , họ phải dựa vào trợ_cấp xã_hội và vay_mượn gia_đình , bạn_bè .
Trong thời_gian chờ_đợi , hành_khách được hãng phục_vụ ăn_uống theo quy_định kèm bồi_thường thiện_chí 300.000 đồng / hành_khách .
Đề_tài " Cải_thiện chất_lượng giống lúa Một bụi đỏ Hồng_Dân " của ông Võ_Đăng_Ký được hội_đồng tư_vấn xét_duyệt đề_cương tỉnh Bạc_Liêu duyệt và cho triển_khai thực_hiện từ tháng 9-2018 .
Đồng_thời NH HDBank cũng cho_biết đã khen_thưởng nhân_viên và bảo_vệ ngân_hàng vì đã dũng_cảm đối_mặt với tên cướp có vũ_khí và chất_nổ để bảo_vệ tài_sản của Ngân_hàng , tính_mạng của cán_bộ , nhân_viên tại quầy giao_dịch .
Ảnh : Albermarle - Charlottesville_Regional_Jail .
Ứng_viên phải trải qua hai vòng khám .
Đặc_biệt cư_dân lo_lắng nhất về hệ_thống phò

In [ ]:
df_train_pre = pd.DataFrame(df_train_sumary, columns = ['predict'])
bigdata_train = pd.concat([df_train_pre.reset_index(drop=True), train_df.original.reset_index(drop=True)], axis=1)
bigdata_train = pd.concat([train_df['file'].reset_index(drop=True), bigdata.reset_index(drop=True)], axis=1)

## Kết quả và đánh giá

In [ ]:
from rouge_score import rouge_scorer
score_list_train = []
for i in range(len(bigdata_train.original)):
  scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
  scores = scorer.score(bigdata_train.predict[i],bigdata_train.original[i])
  score_list_train.append(scores)

In [ ]:
score_r1 = []
score_rl = []
score_r2 = []
for i in range(len(score_list)):
  score_r1.append(score_list_train[i]['rouge1'].fmeasure)
  score_r2.append(score_list_train[i]['rouge2'].fmeasure)
  score_rl.append(score_list_train[i]['rougeL'].fmeasure)

In [ ]:
import statistics
print(statistics.mean(score_r1))
print(statistics.mean(score_r2))
print(statistics.mean(score_rl))